In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
import numpy as np 
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree   import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [3]:
x_test = pd.read_csv('x_test.csv', header=None)
x_train = pd.read_csv('x_train.csv', header=None)
y_test_baseline = pd.read_csv('y_test_baseline.csv', header=None)
y_train = pd.read_csv('y_train.csv', header=None)

In [4]:
print("x_train: ",x_train.shape)
print("y_train: ",y_train.shape)
print("x_test: ",x_test.shape)
print("y_test_baseline: ",y_test_baseline.shape)

x_train:  (406709, 55)
y_train:  (406709, 2)
x_test:  (174305, 55)
y_test_baseline:  (174305, 2)


In [5]:
y_train.values[1:,1].shape

(406708,)

In [12]:
x_train.values[2:,:].astype(int)

array([[     1,   2764,     39, ...,      0,      0,      0],
       [     2,   2773,    158, ...,      0,      0,      0],
       [     3,   3083,    108, ...,      0,      0,      0],
       ...,
       [406705,   2715,     73, ...,      0,      0,      0],
       [406706,   3020,    179, ...,      0,      0,      0],
       [406707,   3368,    191, ...,      1,      0,      0]])

In [7]:
def train_using_gini(X_train, y_train):
    # Gini index will prefer attributes where entropy is lower
    # Gini Index is a metric to measure how often a randomly
    # chosen element would be incorrectly identified.
    # Classifier obj
    clf_gini = DecisionTreeClassifier(criterion="gini",
        random_state=100, max_depth=3, min_samples_leaf=5)

    #train
    clf_gini.fit(X_train, y_train)
    return clf_gini

In [16]:

def prediction(X_test, clf_object):
    y_pred = clf_object.predict(X_test)
    print("Predicted values: ")
    print(y_pred)
    return y_pred
def calculate_Acc(y_test, y_pred):
    print("Confusion matrix: \n")
    confusion_matrix(y_test, y_pred)
    print("Accuracy: ", accuracy_score(y_test,y_pred)*100)
    print("Report ", classification_report(y_test, y_pred))

In [14]:
clf_gini = train_using_gini(x_train.values[1:,:].astype(int), y_train.values[1:,1].astype(int))

In [17]:
  # Operational Phase 
print("Results Using Gini Index:") 

Results Using Gini Index:


In [19]:
     # Prediction using gini 
y_pred_gini = prediction(x_test.values[1:,:].astype(int), clf_gini)
calculate_Acc(y_test_baseline.values[1:,1].astype(int), y_pred_gini)

Predicted values: 
[2 1 2 ... 7 2 2]


NameError: name 'y_test' is not defined